In [1]:
"""
Creamos el dataset.
1.- Extraemos frames de un video
2.- Detectamos rostros en los frames
3.- Cramos las mascaras de los rostros
"""

'\nCreamos el dataset.\n1.- Extraemos frames de un video\n2.- Detectamos rostros en los frames\n3.- Cramos las mascaras de los rostros\n'

In [2]:
import cv2
import dlib
import numpy as np
import os
from os.path import isfile, isdir, exists

In [3]:
"""
Referencia: https://github.com/gallardorafael/edge2art
Extrae cada frame de un video y guardas las imagenes
"""
EVERY_FRAMES = 5
def extraer_frames(file_input, dir_output):
        # Validamos la existencia del video
        if isfile(file_input):
            video_path = file_input
        else:
            print("No existe el video.")
            return
        
        #Definición del CODEC
        cap = cv2.VideoCapture(file_input)
        if not exists(dir_output):
            os.mkdir(dir_output)

        # OpenCV
        contador = 0
        while cap.isOpened():
            # Obtenemos el frame.
            ret, frame = cap.read()
            if ret == True:
                if contador % EVERY_FRAMES == 0:
                    data_path = dir_output + "/" + str(contador) + ".png"
                    print("Extrayendo frame ", contador, " en ", data_path)
                    cv2.imwrite(data_path, frame)
                contador += 1
            else:
                break
        cap.release()

In [4]:
"""
OpenCV’s deep learning face detector esta basado en el
framework Single Shot Detector (SSD) con base en una red
ResNet 
"""

# Para el recorte
CROP_SIZE = 256

# Cargamos modelo de la red
path_model="./deploy.proto.txt"
path_weigths="./res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(path_model, path_weigths)
MIN_CONFIDENCE = 0.8 # Probabilidad minima para la detección

def detectar_rostro(image):
    # Cargamos la imagen y construimos una entraba para la red
    # reescalamos a 300x300 y la normalizamos
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    # Pasamos la entrada a la red y obtenemos las detecciones y 
    # predicciones
    net.setInput(blob)
    detections = net.forward()

    # Iteramos sobre las detecciones
    for i in range(0, detections.shape[2]):
        # Extraemos la probabilidad asociada a una prediccion
        confidence = detections[0, 0, i, 2]

        # Comparamos las probabilidades y verificamos que la actual 
        # sea mayor a la minima, para asegurar que sí es correcta
        # la detección
        if confidence > MIN_CONFIDENCE:
            # Calculamos las coordenadas (x, y) para el rectangulo que
            # enmarque el rostro
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            y = startY - 10 if startY - 10 > 10 else startY + 10
            distanceX = abs(endX-startX)
            distanceY = abs(endY-startY)
            startY -= distanceY * 0.2
            endY += distanceY * 0.1
            endX += distanceX * 0.5
            startX -= distanceX * 0.5
            startX = int(max(startX,0))
            startY = int(max(startY,0))
            endX = int(min(endX,w))
            endY = int(min(endY,h))
            image = image[startY:endY,startX:endX,:]
        return cv2.resize(image,(CROP_SIZE, CROP_SIZE))

![./imgs/shape_predictor_68_face_landmarks.png](./imgs/shape_predictor_68_face_landmarks.png)

In [5]:
"""
The Face Landmark Detection algorithm offered by Dlib is an
implementation of the Ensemble of Regression Trees (ERT) 
presented in 2014 by Kazemi and Sullivan. 

This technique utilize simple and fast feature (pixel 
intensities differences) to directly estimate the landmark 
positions.

These estimated positions are subsequently refined with an 
iterative process done by a cascade of regressors. The 
regressors produces a new estimate from the previous one, 
trying to reduce the alignment error of the estimated points 
at each iteration.

The algorithm is blazing fast, in fact it takes about 1–3ms
(on desktop platform) to detect (align) a set of 68 
landmarks on a given face.
"""

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")


def crear_mascara(image):
    
    try:
        image2 = image[:,:,:].copy()
    except TypeError:
        raise
        
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = gray
    rects = detector(gray, 1)
    
    for (i, rect) in enumerate(rects):

        if(i > 0):
            break  
        
        features = predictor(gray, rect)
        
        face = []
        eye1 = []
        eye2 = []
        eye1u = []
        eye2u = []
        nose = []
        mouth = []
        mouth2 = []
        
        for b in range(68):
            p1 = [features.part(b).x, features.part(b).y]
            
            if(b < 17):
                face.append(p1)
            
            if(b >= 22 and b <= 26):
                eye1u.append(p1) 
            
            if(b >= 17 and b <= 21):
                eye2u.append(p1)   
            
            if(b >= 36 and b <= 41):
                eye1.append(p1)
            
            if(b >= 27 and b <= 35):
                nose.append(p1)  
            
            if(b >= 42 and b <= 47):
                eye2.append(p1)
            
            if(b >= 48 and b <= 59):
                mouth.append(p1) 
            
            if(b >= 60 and b <= 68):
                mouth2.append(p1)
        
        b_image = np.zeros_like(image)
        
        cv2.fillPoly(b_image, [np.array(face)], (0,255,0))
        cv2.fillPoly(b_image, [np.array(eye1)], (255,255,0))
        cv2.fillPoly(b_image, [np.array(eye2)], (255,255,0))
        
        
        cv2.fillPoly(image, [np.array(eye1)], (255,255,0))
        cv2.fillPoly(image, [np.array(eye2)], (255,255,0))
        
        
        cv2.fillPoly(b_image, [np.array(nose)], (255,0,255))
        cv2.fillPoly(b_image, [np.array(mouth)], (0,0,0))
        cv2.fillPoly(b_image, [np.array(eye1u)], (0,255,255))
        cv2.fillPoly(b_image, [np.array(eye2u)], (0,255,255))
        cv2.fillPoly(b_image, [np.array(mouth2)],(255,0,0))
        mask = cv2.inRange(image, (255,255,0), (255,255,0))
       
        mask = cv2.bitwise_and(image2, image2, mask = mask)
        G = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask1 = cv2.threshold(G, 100, 250, cv2.THRESH_BINARY)
        mask1 = cv2.bitwise_and(255 * np.ones_like(image), 255 * np.ones_like(image),mask = mask1[1])
        mask = cv2.bitwise_or(b_image, mask1)
        
    return mask

In [6]:
def detectar_rostros_en_frames(dir_input, dir_output):
    count = 0
    # Se crea el directorio de salida
    if not isdir(dir_output):
        os.mkdir(dir_output)
    for image_name in os.listdir(dir_input):
        count += 1
        in_path = os.path.join(dir_input, image_name)
        out_path = os.path.join(dir_output, image_name)
        print("Detectando rostro ", count, " en ", in_path)
        image = cv2.imread(in_path)
        image = detectar_rostro(image)
        cv2.imwrite(in_path, image)
        print("Creando mascara ", count, " en ", out_path)
        image = crear_mascara(image)
        cv2.imwrite(out_path, image)

In [7]:
file_input = "./video1.mp4"
dir_output =  "./tagged_train"
extraer_frames(file_input, dir_output)

file_input = "./video2.mp4"
dir_output =  "./tagged_test"
extraer_frames(file_input, dir_output)

Extrayendo frame  0  en  ./tagged_train/0.png
Extrayendo frame  5  en  ./tagged_train/5.png
Extrayendo frame  10  en  ./tagged_train/10.png
Extrayendo frame  15  en  ./tagged_train/15.png
Extrayendo frame  20  en  ./tagged_train/20.png
Extrayendo frame  25  en  ./tagged_train/25.png
Extrayendo frame  30  en  ./tagged_train/30.png
Extrayendo frame  35  en  ./tagged_train/35.png
Extrayendo frame  40  en  ./tagged_train/40.png
Extrayendo frame  45  en  ./tagged_train/45.png
Extrayendo frame  50  en  ./tagged_train/50.png
Extrayendo frame  55  en  ./tagged_train/55.png
Extrayendo frame  60  en  ./tagged_train/60.png
Extrayendo frame  65  en  ./tagged_train/65.png
Extrayendo frame  70  en  ./tagged_train/70.png
Extrayendo frame  75  en  ./tagged_train/75.png
Extrayendo frame  80  en  ./tagged_train/80.png
Extrayendo frame  85  en  ./tagged_train/85.png
Extrayendo frame  90  en  ./tagged_train/90.png
Extrayendo frame  95  en  ./tagged_train/95.png
Extrayendo frame  100  en  ./tagged_train/10

Extrayendo frame  825  en  ./tagged_train/825.png
Extrayendo frame  830  en  ./tagged_train/830.png
Extrayendo frame  835  en  ./tagged_train/835.png
Extrayendo frame  840  en  ./tagged_train/840.png
Extrayendo frame  845  en  ./tagged_train/845.png
Extrayendo frame  850  en  ./tagged_train/850.png
Extrayendo frame  855  en  ./tagged_train/855.png
Extrayendo frame  860  en  ./tagged_train/860.png
Extrayendo frame  865  en  ./tagged_train/865.png
Extrayendo frame  870  en  ./tagged_train/870.png
Extrayendo frame  875  en  ./tagged_train/875.png
Extrayendo frame  880  en  ./tagged_train/880.png
Extrayendo frame  885  en  ./tagged_train/885.png
Extrayendo frame  890  en  ./tagged_train/890.png
Extrayendo frame  895  en  ./tagged_train/895.png
Extrayendo frame  900  en  ./tagged_train/900.png
Extrayendo frame  905  en  ./tagged_train/905.png
Extrayendo frame  910  en  ./tagged_train/910.png
Extrayendo frame  915  en  ./tagged_train/915.png
Extrayendo frame  920  en  ./tagged_train/920.png


Extrayendo frame  1630  en  ./tagged_train/1630.png
Extrayendo frame  1635  en  ./tagged_train/1635.png
Extrayendo frame  1640  en  ./tagged_train/1640.png
Extrayendo frame  1645  en  ./tagged_train/1645.png
Extrayendo frame  1650  en  ./tagged_train/1650.png
Extrayendo frame  1655  en  ./tagged_train/1655.png
Extrayendo frame  1660  en  ./tagged_train/1660.png
Extrayendo frame  1665  en  ./tagged_train/1665.png
Extrayendo frame  1670  en  ./tagged_train/1670.png
Extrayendo frame  1675  en  ./tagged_train/1675.png
Extrayendo frame  1680  en  ./tagged_train/1680.png
Extrayendo frame  1685  en  ./tagged_train/1685.png
Extrayendo frame  1690  en  ./tagged_train/1690.png
Extrayendo frame  1695  en  ./tagged_train/1695.png
Extrayendo frame  1700  en  ./tagged_train/1700.png
Extrayendo frame  1705  en  ./tagged_train/1705.png
Extrayendo frame  1710  en  ./tagged_train/1710.png
Extrayendo frame  1715  en  ./tagged_train/1715.png
Extrayendo frame  1720  en  ./tagged_train/1720.png
Extrayendo f

Extrayendo frame  625  en  ./tagged_test/625.png
Extrayendo frame  630  en  ./tagged_test/630.png
Extrayendo frame  635  en  ./tagged_test/635.png
Extrayendo frame  640  en  ./tagged_test/640.png
Extrayendo frame  645  en  ./tagged_test/645.png
Extrayendo frame  650  en  ./tagged_test/650.png
Extrayendo frame  655  en  ./tagged_test/655.png
Extrayendo frame  660  en  ./tagged_test/660.png
Extrayendo frame  665  en  ./tagged_test/665.png
Extrayendo frame  670  en  ./tagged_test/670.png
Extrayendo frame  675  en  ./tagged_test/675.png
Extrayendo frame  680  en  ./tagged_test/680.png
Extrayendo frame  685  en  ./tagged_test/685.png
Extrayendo frame  690  en  ./tagged_test/690.png
Extrayendo frame  695  en  ./tagged_test/695.png
Extrayendo frame  700  en  ./tagged_test/700.png
Extrayendo frame  705  en  ./tagged_test/705.png
Extrayendo frame  710  en  ./tagged_test/710.png
Extrayendo frame  715  en  ./tagged_test/715.png
Extrayendo frame  720  en  ./tagged_test/720.png
Extrayendo frame  72

Extrayendo frame  1450  en  ./tagged_test/1450.png
Extrayendo frame  1455  en  ./tagged_test/1455.png
Extrayendo frame  1460  en  ./tagged_test/1460.png
Extrayendo frame  1465  en  ./tagged_test/1465.png
Extrayendo frame  1470  en  ./tagged_test/1470.png
Extrayendo frame  1475  en  ./tagged_test/1475.png
Extrayendo frame  1480  en  ./tagged_test/1480.png
Extrayendo frame  1485  en  ./tagged_test/1485.png
Extrayendo frame  1490  en  ./tagged_test/1490.png
Extrayendo frame  1495  en  ./tagged_test/1495.png
Extrayendo frame  1500  en  ./tagged_test/1500.png
Extrayendo frame  1505  en  ./tagged_test/1505.png
Extrayendo frame  1510  en  ./tagged_test/1510.png
Extrayendo frame  1515  en  ./tagged_test/1515.png
Extrayendo frame  1520  en  ./tagged_test/1520.png
Extrayendo frame  1525  en  ./tagged_test/1525.png
Extrayendo frame  1530  en  ./tagged_test/1530.png
Extrayendo frame  1535  en  ./tagged_test/1535.png
Extrayendo frame  1540  en  ./tagged_test/1540.png
Extrayendo frame  1545  en  ./t

In [8]:
dir_input = "./tagged_train"
dir_output =  "./input_train"
detectar_rostros_en_frames(dir_input, dir_output)

dir_input = "./tagged_test"
dir_output =  "./input_test"
detectar_rostros_en_frames(dir_input, dir_output)

Detectando rostro  1  en  ./tagged_train/210.png
Creando mascara  1  en  ./input_train/210.png
Detectando rostro  2  en  ./tagged_train/525.png
Creando mascara  2  en  ./input_train/525.png
Detectando rostro  3  en  ./tagged_train/295.png
Creando mascara  3  en  ./input_train/295.png
Detectando rostro  4  en  ./tagged_train/575.png
Creando mascara  4  en  ./input_train/575.png
Detectando rostro  5  en  ./tagged_train/935.png
Creando mascara  5  en  ./input_train/935.png
Detectando rostro  6  en  ./tagged_train/355.png
Creando mascara  6  en  ./input_train/355.png
Detectando rostro  7  en  ./tagged_train/540.png
Creando mascara  7  en  ./input_train/540.png
Detectando rostro  8  en  ./tagged_train/1080.png
Creando mascara  8  en  ./input_train/1080.png
Detectando rostro  9  en  ./tagged_train/1315.png
Creando mascara  9  en  ./input_train/1315.png
Detectando rostro  10  en  ./tagged_train/690.png
Creando mascara  10  en  ./input_train/690.png
Detectando rostro  11  en  ./tagged_train/56

Creando mascara  85  en  ./input_train/585.png
Detectando rostro  86  en  ./tagged_train/305.png
Creando mascara  86  en  ./input_train/305.png
Detectando rostro  87  en  ./tagged_train/185.png
Creando mascara  87  en  ./input_train/185.png
Detectando rostro  88  en  ./tagged_train/1385.png
Creando mascara  88  en  ./input_train/1385.png
Detectando rostro  89  en  ./tagged_train/950.png
Creando mascara  89  en  ./input_train/950.png
Detectando rostro  90  en  ./tagged_train/980.png
Creando mascara  90  en  ./input_train/980.png
Detectando rostro  91  en  ./tagged_train/1185.png
Creando mascara  91  en  ./input_train/1185.png
Detectando rostro  92  en  ./tagged_train/175.png
Creando mascara  92  en  ./input_train/175.png
Detectando rostro  93  en  ./tagged_train/1805.png
Creando mascara  93  en  ./input_train/1805.png
Detectando rostro  94  en  ./tagged_train/695.png
Creando mascara  94  en  ./input_train/695.png
Detectando rostro  95  en  ./tagged_train/430.png
Creando mascara  95  en 

Creando mascara  168  en  ./input_train/1695.png
Detectando rostro  169  en  ./tagged_train/1630.png
Creando mascara  169  en  ./input_train/1630.png
Detectando rostro  170  en  ./tagged_train/270.png
Creando mascara  170  en  ./input_train/270.png
Detectando rostro  171  en  ./tagged_train/1055.png
Creando mascara  171  en  ./input_train/1055.png
Detectando rostro  172  en  ./tagged_train/655.png
Creando mascara  172  en  ./input_train/655.png
Detectando rostro  173  en  ./tagged_train/1600.png
Creando mascara  173  en  ./input_train/1600.png
Detectando rostro  174  en  ./tagged_train/405.png
Creando mascara  174  en  ./input_train/405.png
Detectando rostro  175  en  ./tagged_train/365.png
Creando mascara  175  en  ./input_train/365.png
Detectando rostro  176  en  ./tagged_train/1680.png
Creando mascara  176  en  ./input_train/1680.png
Detectando rostro  177  en  ./tagged_train/1370.png
Creando mascara  177  en  ./input_train/1370.png
Detectando rostro  178  en  ./tagged_train/910.png

Creando mascara  251  en  ./input_train/605.png
Detectando rostro  252  en  ./tagged_train/1815.png
Creando mascara  252  en  ./input_train/1815.png
Detectando rostro  253  en  ./tagged_train/505.png
Creando mascara  253  en  ./input_train/505.png
Detectando rostro  254  en  ./tagged_train/720.png
Creando mascara  254  en  ./input_train/720.png
Detectando rostro  255  en  ./tagged_train/805.png
Creando mascara  255  en  ./input_train/805.png
Detectando rostro  256  en  ./tagged_train/1470.png
Creando mascara  256  en  ./input_train/1470.png
Detectando rostro  257  en  ./tagged_train/1060.png
Creando mascara  257  en  ./input_train/1060.png
Detectando rostro  258  en  ./tagged_train/1715.png
Creando mascara  258  en  ./input_train/1715.png
Detectando rostro  259  en  ./tagged_train/1575.png
Creando mascara  259  en  ./input_train/1575.png
Detectando rostro  260  en  ./tagged_train/1625.png
Creando mascara  260  en  ./input_train/1625.png
Detectando rostro  261  en  ./tagged_train/455.pn

Creando mascara  333  en  ./input_train/1065.png
Detectando rostro  334  en  ./tagged_train/215.png
Creando mascara  334  en  ./input_train/215.png
Detectando rostro  335  en  ./tagged_train/1090.png
Creando mascara  335  en  ./input_train/1090.png
Detectando rostro  336  en  ./tagged_train/1235.png
Creando mascara  336  en  ./input_train/1235.png
Detectando rostro  337  en  ./tagged_train/160.png
Creando mascara  337  en  ./input_train/160.png
Detectando rostro  338  en  ./tagged_train/25.png
Creando mascara  338  en  ./input_train/25.png
Detectando rostro  339  en  ./tagged_train/660.png
Creando mascara  339  en  ./input_train/660.png
Detectando rostro  340  en  ./tagged_train/1770.png
Creando mascara  340  en  ./input_train/1770.png
Detectando rostro  341  en  ./tagged_train/830.png
Creando mascara  341  en  ./input_train/830.png
Detectando rostro  342  en  ./tagged_train/1115.png
Creando mascara  342  en  ./input_train/1115.png
Detectando rostro  343  en  ./tagged_train/1520.png
Cr

Detectando rostro  51  en  ./tagged_test/1195.png
Creando mascara  51  en  ./input_test/1195.png
Detectando rostro  52  en  ./tagged_test/560.png
Creando mascara  52  en  ./input_test/560.png
Detectando rostro  53  en  ./tagged_test/835.png
Creando mascara  53  en  ./input_test/835.png
Detectando rostro  54  en  ./tagged_test/200.png
Creando mascara  54  en  ./input_test/200.png
Detectando rostro  55  en  ./tagged_test/1230.png
Creando mascara  55  en  ./input_test/1230.png
Detectando rostro  56  en  ./tagged_test/1075.png
Creando mascara  56  en  ./input_test/1075.png
Detectando rostro  57  en  ./tagged_test/860.png
Creando mascara  57  en  ./input_test/860.png
Detectando rostro  58  en  ./tagged_test/760.png
Creando mascara  58  en  ./input_test/760.png
Detectando rostro  59  en  ./tagged_test/375.png
Creando mascara  59  en  ./input_test/375.png
Detectando rostro  60  en  ./tagged_test/250.png
Creando mascara  60  en  ./input_test/250.png
Detectando rostro  61  en  ./tagged_test/149

Detectando rostro  136  en  ./tagged_test/580.png
Creando mascara  136  en  ./input_test/580.png
Detectando rostro  137  en  ./tagged_test/840.png
Creando mascara  137  en  ./input_test/840.png
Detectando rostro  138  en  ./tagged_test/880.png
Creando mascara  138  en  ./input_test/880.png
Detectando rostro  139  en  ./tagged_test/105.png
Creando mascara  139  en  ./input_test/105.png
Detectando rostro  140  en  ./tagged_test/1710.png
Creando mascara  140  en  ./input_test/1710.png
Detectando rostro  141  en  ./tagged_test/1450.png
Creando mascara  141  en  ./input_test/1450.png
Detectando rostro  142  en  ./tagged_test/680.png
Creando mascara  142  en  ./input_test/680.png
Detectando rostro  143  en  ./tagged_test/555.png
Creando mascara  143  en  ./input_test/555.png
Detectando rostro  144  en  ./tagged_test/300.png
Creando mascara  144  en  ./input_test/300.png
Detectando rostro  145  en  ./tagged_test/1325.png
Creando mascara  145  en  ./input_test/1325.png
Detectando rostro  146  

Creando mascara  220  en  ./input_test/1675.png
Detectando rostro  221  en  ./tagged_test/845.png
Creando mascara  221  en  ./input_test/845.png
Detectando rostro  222  en  ./tagged_test/20.png
Creando mascara  222  en  ./input_test/20.png
Detectando rostro  223  en  ./tagged_test/345.png
Creando mascara  223  en  ./input_test/345.png
Detectando rostro  224  en  ./tagged_test/1475.png
Creando mascara  224  en  ./input_test/1475.png
Detectando rostro  225  en  ./tagged_test/960.png
Creando mascara  225  en  ./input_test/960.png
Detectando rostro  226  en  ./tagged_test/1030.png
Creando mascara  226  en  ./input_test/1030.png
Detectando rostro  227  en  ./tagged_test/1295.png
Creando mascara  227  en  ./input_test/1295.png
Detectando rostro  228  en  ./tagged_test/440.png
Creando mascara  228  en  ./input_test/440.png
Detectando rostro  229  en  ./tagged_test/70.png
Creando mascara  229  en  ./input_test/70.png
Detectando rostro  230  en  ./tagged_test/1570.png
Creando mascara  230  en  

Detectando rostro  305  en  ./tagged_test/1460.png
Creando mascara  305  en  ./input_test/1460.png
Detectando rostro  306  en  ./tagged_test/1645.png
Creando mascara  306  en  ./input_test/1645.png
Detectando rostro  307  en  ./tagged_test/190.png
Creando mascara  307  en  ./input_test/190.png
Detectando rostro  308  en  ./tagged_test/1725.png
Creando mascara  308  en  ./input_test/1725.png
Detectando rostro  309  en  ./tagged_test/885.png
Creando mascara  309  en  ./input_test/885.png
Detectando rostro  310  en  ./tagged_test/1265.png
Creando mascara  310  en  ./input_test/1265.png
Detectando rostro  311  en  ./tagged_test/500.png
Creando mascara  311  en  ./input_test/500.png
Detectando rostro  312  en  ./tagged_test/395.png
Creando mascara  312  en  ./input_test/395.png
Detectando rostro  313  en  ./tagged_test/640.png
Creando mascara  313  en  ./input_test/640.png
Detectando rostro  314  en  ./tagged_test/970.png
Creando mascara  314  en  ./input_test/970.png
Detectando rostro  315